# Phenotype Formatting

This module implements a workflow used to format phenotype data to prepare it for cis TensorQTL analysis.

## Input

1. `protocol_example.protein.bed.gz`

## Output

1. `protocol_example.protein.bed.chr21.bed.gz`
2. `protocol_example.protein.bed.chr22.bed.gz`
3. `protocol_example.protein.bed.phenotype_by_chrom_files.txt`

## Minimal Working Example

The proteomics data used in this MWE can be found on [synapse](https://www.synapse.org/#!Synapse:syn52369482).

### Step 1: Partition by Chromosome
Timing: < 1 minute

In [ ]:
sos run pipeline/phenotype_formatting.ipynb phenotype_by_chrom \
    --cwd output/phenotype_by_chrom \
    --phenoFile output/phenotype/protocol_example.protein.bed.gz \
    --chrom `for i in {21..22}; do echo chr$i; done` \
    --container singularity/bioinfo.sif

```
INFO: Running phenotype_by_chrom_1:
INFO: phenotype_by_chrom_1 (index=0) is completed.
INFO: phenotype_by_chrom_1 (index=1) is completed.
INFO: phenotype_by_chrom_1 output:   output/phenotype_by_chrom/protocol_example.protein.bed.chr22.bed.gz output/phenotype_by_chrom/protocol_example.protein.bed.chr21.bed.gz in 2 groups
INFO: Running phenotype_by_chrom_2:
INFO: phenotype_by_chrom_2 is completed.
INFO: phenotype_by_chrom_2 output:   output/phenotype_by_chrom/protocol_example.protein.bed.phenotype_by_chrom_files.txt
INFO: Workflow phenotype_by_chrom (ID=w39bcae3df05196d5) is executed successfully with 2 completed steps and 3 completed substeps.
```

## Command interface

In [ ]:
sos run phenotype_formatting.ipynb -h

In [ ]:
usage: sos run phenotype_formatting.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  phenotype_by_chrom
  phenotype_by_region
  bam_subsetting
  gct_extract_samples

Global Workflow Options:
  --cwd output (as path)
                        Work directory & output directory
  --container ''
                        The filename namefor output data
  --entrypoint {('micromamba run -a "" -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}

  --job-size 1 (as int)
                        For cluster jobs, number commands to run per job
  --walltime 5h
                        Wall clock time expected
  --mem 16G
                        Memory expected
  --numThreads 20 (as int)
                        Number of threads
  --phenoFile VAL (as path, required)
                        Path to the input molecular phenotype data.
  --name  f'{phenoFile:bn}'

                        name for the analysis output

Sections
  phenotype_by_chrom_1:
    Workflow Options:
      --chrom VAL VAL ... (as type, required)
                        list of chroms to extract
  phenotype_by_chrom_2:
  phenotype_by_region_1:
    Workflow Options:
      --region-list VAL (as path, required)
                        An index text file with 4 columns specifying the chr,
                        start, end and name of regions to analyze
  phenotype_by_region_2:
  bam_subsetting:
    Workflow Options:
      --region VAL VAL ... (as type, required)
                        Input to `samtools view` coordinates, for example,
                        --region chr21 chr22
  gct_extract_samples:  Extract samples from expression data generated by
                        RNASeQC
    Workflow Options:
      --keep-samples VAL (as path, required)

### Step 1: Partition by chromosome

In [ ]:
[phenotype_by_chrom_1]
# list of chroms to extract 
parameter: chrom = list
chrom = list(set(chrom))
# Path to the input molecular phenotype data.
input: phenoFile, for_each = "chrom"
output: f'{cwd}/{name}.{_chrom}.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime,  mem = mem, tags = f'{step_name}_{_output:bn}'  
bash: expand = "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout',container = container, entrypoint=entrypoint
    zcat $[_input] | head -1 > $[_output:n]
    tabix $[_input] $[_chrom] >> $[_output:n] 
    bgzip -f $[_output:n]
    tabix -p bed $[_output] -f
bash: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint=entrypoint
        stdout=$[_output:n].stdout
        for i in $[_output] ; do 
        echo "output_info: $i " >> $stdout;
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "`   >> $stdout;
        echo "output_rows:" `zcat $i | wc -l  | cut -f 1 -d " "`   >> $stdout;
        echo "output_column:" `zcat $i | grep -v "##"   | head -1 | wc -w `   >> $stdout;
        echo "output_headerow:" `zcat $i | grep "##" | wc -l `   >> $stdout;
        echo "output_preview:"   >> $stdout;
        zcat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6   >> $stdout ; done

In [ ]:
[phenotype_by_chrom_2]
# Path to the input molecular phenotype data.
input: group_by = "all"
output: f'{cwd}/{name}.{step_name[:-2]}_files.txt'
import pandas as pd
chrom = [str(x).split(".")[-3].replace("chr","") for x in _input]
chrom_df = pd.DataFrame({"#id" : chrom ,"#dir" : _input})
chrom_df.to_csv(_output,index = 0,sep = "\t")